# Projet Final - GBM8770
## Multi-Scale Line Detector

In [1]:
%load_ext autoreload
%autoreload 2

# Importe les modules qui seront utilisé dans le laboratoire.
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
import skimage.morphology as skmorph
%matplotlib inline
# Étend la taille des figures
plt.rcParams["figure.figsize"] = (18, 8)

In [2]:
# Importe la classe MSLD et la fonction load_dataset définie dans le fichier MSLD.py
from msld import MSLD, load_dataset

# 1. Implémentation de la MSLD

## 1.1. Préparatifs
Dans un soucis d’optimisation de l’implémentation, nous allons utiliser des convolutions dès que cela est possible. Particulièrement pour le calcul des moyennes d’intensités sur une fenêtre de taille `W` et le long des lignes de longueur `L`. De plus ces masques ne seront calculés qu’une seule fois, lors de l’instanciation de la classe MSLD.

**Q1.** Donnez la valeur du masque permettant le calcul de la valeur moyenne d’une fenêtre de taille `W`. Complétez le constructeur de la classe `MSLD` en définissant `avg_mask`.


In [ ]:
# Décrire en mots le masque qu'on construit 
# nbre d'orientation = 12 

**Q2.** Expliquez comment construire les masques permettant le calcul de la valeur moyenne le long de lignes de taille `L` pour `n_orientation` par rotations successives. Complétez le constructeur de la classe `MSLD` avec la définition de `line_detectors_masks`.


In [ ]:
# Vous pouvez dès à présent instancier l’objet msld en remplaçant les hyper-paramètres W, L et n_orientation par leurs
# valeurs conseillées par l’article.
msld = MSLD(W=..., L=..., n_orientation=...)

Pour entraîner et valider l’algorithme nous allons utiliser la base de données DRIVE fournie dans l'archive.


**Q3.** Complétez la fonction `load_dataset()` pour qu’elle charge et renvoie la base de données d’entraînement et de test (en incluant pour chaque échantillon: l’image, le label et le masque). Puis affichez l’image, le label et le masque de la première image de l’ensemble d’entraînement.

*(On notera cette image I1.)*

Ne pas utiliser deepcopy dans cette fonction / numéro

In [ ]:
train, test = load_dataset()
I1 = train[0]
# I1['image'] contient l'image I1, 
# I1['label'] contient la segmentation visée, 
# I1['mask'] contient le masque

## 1.2. Basic Line Detector

**Q1.** À l’aide de la section 3.1 de l’article, implémentez `basic_line_detector(grey_lvl, L)`.

**Q2.** Affichez la réponse du filtre appliqué à l’image I1 pour les longueurs L=1 et L=15. Comparez les deux et commentez.   

*Note: Attention le filtre BLD ne s’applique pas à tous les canaux de l’image. Relisez la section 3.1 de l’article pour plus d’informations...*

In [ ]:
grey_lvl_I1 = ...
I1_bld_L1 = msld.basic_line_detector(grey_lvl_I1, L=1)
I1_bld_L15 = msld.basic_line_detector(grey_lvl_I1, L=15)

## 1.3. Multi-Scale Line Detector

**Q1.** À l’aide des sections 3.2 et 3.3 de l’article et de la méthode `self.basic_line_detector(grey_lvl, L)` déjà implémentée, implémentez `multi_scale_line_detector(image)`.

**Q1.** Affichez le résultat de l’algorithme appliqué à l’image I1. Comparez avec les réponses du filtre obtenues à la question 1.2.2, et commentez.

In [ ]:
I1_msld = msld.multi_scale_line_detector(I1)

## 1.4. Apprentissage du seuil
 
Pour calculer le seuil donnant la meilleure précision sur l’ensemble d’entraînement efficacement, nous allons utiliser la courbe [ROC (Receiver Operating Characteristic)](https://en.wikipedia.org/wiki/Receiver_operating_characteristic). Cette courbe sera étudiée en détail dans la partie 2. Pour le moment, il vous faut juste savoir qu’elle associe à chaque seuil possible, le taux de faux positifs et le taux de vrais positifs si ce seuil était choisi.

**Q1.** Donnez la formule de la précision ACC (*accuracy* en anglais) en fonction du taux de faux positifs FPR, du taux de vrais positifs TPR, ainsi que du nombre de valeurs positives P, de valeurs négatives N et du nombre total de pixels S de la région d'intéret.   

Utilisez la syntaxe $\LaTeX$ pour formatter l'équation.

*(Vous pouvez vous aider des formules indiqués sur la page Wikipédia de la courbe ROC.)*

Pour vous faire gagner du temps, on vous donne la méthode `msld.roc(dataset)` qui calcule le TPR et le FPR associé à chaque seuil possible pour un dataset donné. Cette méthode utilise la fonction [`roc_curve`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_curve.html) de `sklearn`. Assurez-vous que vous comprenez bien chaque ligne du code.

**Q2.** Pourquoi est-il si important de ne sélectionner uniquement les pixels qui appartiennent au masque. En particulier, comment évoluerait la précision si on sélectionnait finalement tous les pixels de l'image (sachant que hors du masque la prédiction du MSLD est toujours nulle)?

**Q3.** À l’aide de la méthode `self.roc(dataset)` et de la formule de la question 1.4.1, complétez la méthode `learn_threshold(dataset)` qui identifie le seuil pour laquelle la précision est la plus élevée.

**Q4.** Utilisez cette méthode pour apprendre le seuil sur les images d’entraînement. Pour quelle raison faut il absolument conserver une partie des images (l’ensemble de test) et ne pas les utiliser pendant l’entraînement?

*Note: Il n’est pas anormal d’obtenir un seuil différent de celui annoncé par les auteurs...*

In [ ]:
threshold, accuracy = msld.learn_threshold(train)

## 1.5. Affichage et région d’intérêt

**Q1.** Implémentez la méthode `segment_vessels(image)` qui applique le seuil à la carte de probabilité calculée sur une image. Affichez le résultat de l’algorithme sur l’image I1.


**Q2.**  Vous devriez voir apparaître des erreurs de segmentations aux bords du fond d’oeil. À quoi est
dû ce phénomène?



**Q3.**   Pour palier à ce problème, dupliquez les datasets, érodez les régions d’intérêts (les masques) de 10 pixels, puis effectuez l’apprentissage du seuil à nouveau. Comparez les valeurs du seuil et de la précision globale.



In [ ]:
# Duplique les dataset
train_eroded = deepcopy(train)
test_eroded = deepcopy(test)

for d in train_eroded:
    # Erosion
    d['mask'] = ...

# Même opération pour test_eroded

**Q4.** On vous donne la méthode `show_diff(sample)` qui affiche les faux positifs en rouge, les faux négatifs en bleu, les vrais positifs en blanc et les vrais négatifs en noirs. Avec cette méthode, affichez la différence entre la prédiction et le label sur l’image I1.

# 2. Métriques de segmentation

## 2.1. Précision globale et locale

**Q1.** Implémentez la méthode `naive_metrics(dataset)` qui évalue la précision et la matrice de confusion de l’algorithme pour un dataset donné. Puis calculez ces métriques sur l’ensemble de test.
*(N’oubliez pas de ne sélectionner que les pixels appartenant à la région d’intérêt!)*


**Q2.** Les auteurs proposent une seconde métrique: la précision locale. Quelles raisons avancent-ils
pour motiver cette proposition?


**Q3.** Grâce à une opération morphologique, créez une copie du dataset test tel que son attribut mask corresponde à la région d’intérêt "locale" proposée par les auteurs. Puis, sans modifier la méthode `naive_metrics(dataset)`, calculez la précision et la matrice de confusion locale.


In [ ]:
test_local = deepcopy(test)

...

**Q4.** Cette seconde métrique met plus en valeur l’algorithme proposé que la précision globale. Au vu des erreurs de l’algorithme révélées à la question 1.5.4, donnez une raison supplémentaire (omise par les auteurs) qui explique ce phénomène. 

## 2.2. Indice Dice (bonus)
De nombreux indices existent pour mieux évaluer les performances de segmentation que la précision globale: précision balancée, indice de Jaccard, kappa de Cohen, etc. Nous allons ici utiliser l'indice Sørensen-Dice. En notant $Y$ les labels et $\hat{Y}$ les prédictions de l'algorithme, l'indice Dice est défini par
$$\text{Dice}(Y, \hat{Y}) = 2\dfrac{Y \cap \hat{Y}}{Y + \hat{Y}}.$$

**Q1.** En quoi cette métrique répond aux limitations de la précision globale?

**Q2.** Implémentez la méthode `dice(dataset)` et calculez sa valeur sur la région d’intérêt globale et locale sur l’ensemble de test.


## 2.3 Courbe ROC et aire sous la courbe


Le choix d’un seuil de segmentation revient à faire un compromis entre faux-positifs et faux-négatifs (un seuil élevé limitera les faux-positifs mais augmentera les faux-négatifs et inversement). Le choix du bon compromis dépend bien souvent de l’application. Cependant les algorithmes se distinguent par le choix des caractéristiques à seuiller (ici un filtre MSLD) plutôt que par la méthode de sélection du seuil... [La courbe ROC](https://en.wikipedia.org/wiki/Receiver_operating_characteristic##Area_under_the_curve) permet de représenter les performances de l’algorithme indépendamment du seuil choisi.   

Pour construire cette courbe, on calcule le taux de faux-positifs (FPR) et le taux de vrais-positifs (TPR=1−FNR) pour chaque valeur de seuil. La courbe obtenue (FPR en abscisse, TPR en ordonnée) caractérise l’efficacité de l’algorithme à distinguer les vaisseaux du fond, indépendamment du seuil.

Enfin, pour simplifier la comparaison entre deux courbes ROC, on extrait leurs [aires sous la courbe (AUR)](https://en.wikipedia.org/wiki/Receiver_operating_characteristic##Area_under_the_curve).

**Q1.** Que signifie une AUR de 1, de 0,5 ou de 0 pour les performances du modèle?


**Q2.**  Implémentez la méthode plot_roc(dataset) qui trace la courbe ROC et calcule son AUR. Puis faites de même pour la région d’intérêt globale et pour la région d’intérêt locale. Comparez les résultats et commentez.

# 3. Validation de l’hypothèse de recherche de l’article

À l’aide de votre implémentation de l’algorithme et des différents cas du dataset de test, discutez de la qualité du protocole expérimental pour vérifier les hypothèses de recherche sous-jacentes de l’article.

# 4. Discussion


À l’aide de votre implémentation de l’algorithme et des différents cas du dataset de test, concluez sur
la validité de l’hypothèse de recherche de l’article (tel que vous l’aviez formulé dans le devoir 2).


**Q1.** Discutez du choix des hyper-paramètres `W`, `L` (nombre d’échelles et répartition) et `n_orientation`. Leurs valeurs ont-elles des justifications théoriques? Quels sont leur impact en pratique?

**Q2.** Les auteurs ont choisi le seuil donnant la meilleure précision sur l’ensemble d’entraînement. Connaissant les analyses effectuées dans la partie 2, discutez de ce choix.

**Q3.** En vous appuyant sur vos expérimentations pour les deux dernières parties de cet énoncé, proposez des recommandations pour améliorer l’algorithme.